In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda
import yaml
import pandas as pd
from pycomplexes import convert
import itertools

This tutorial shows how to use the NPT algorithm in complexes. As example an Argon fluid is simulated

# Reference Data

In this tutorial, we set up a NPT-Simlation with a number Ar-Atoms. Every particle acts symbolic as a domain for complexes and is simulated as an independent simulation bead.
In the following you will be provided reference data for a system of Ar-particles.
This system is a basic example of a lennard-jones fluid.

In [ ]:
rho = [0.0025, 0.0059, 0.0102, 0.0128, 0.0153, 0.0178, 0.0204, 0.0229]
p = [74.60, 138.12, 295.96, 448.77, 737.1768776714837, 1269.49, 2216.57, 3816.02]
reference = pd.DataFrame({'rho':rho,'p':p})

In [ ]:
fig, ax = plt.subplots()
ax.plot(reference.p, reference.rho, 'o')
ax.set(xlabel='Pressure [bar]', ylabel='Density [$1/\AA$]')
fig.tight_layout()

# Prepare initial configuration

In [ ]:
# Helper Functions

def make_universe(nbeads):
    """Create MDAnalysis Universe from scratch for manipulation
    """
    u = mda.Universe.empty(nbeads, trajectory=True,
                           n_residues=nbeads,
                           atom_resindex=np.arange(nbeads),
                           n_segments=nbeads,
                           residue_segindex=np.arange(nbeads))
    u.add_TopologyAttr('names', np.array(['A', ] * nbeads))
    u.add_TopologyAttr('resnames', np.array(['AR',]*nbeads))
    u.add_TopologyAttr('resids', np.arange(nbeads) + 1)
    u.add_TopologyAttr('masses', np.ones(nbeads))
    u.add_TopologyAttr('segids', np.array(['A', ] * nbeads))
    u.add_TopologyAttr('altLocs')
    u.add_TopologyAttr('occupancies', np.ones(nbeads))
    u.add_TopologyAttr('icodes')
    u.add_TopologyAttr('tempfactors')
    return u

def density_to_volume(rho, N): # will be in same unit as (1/rho)^3
    """Calculate ocupied volume for a given density and number of particles"""
    return float(N) / rho

def is_cube(n):
    cube = np.round(np.cbrt(n))
    return cube**3 == n

def grid_points(n, boxsize):
    """equally distance points in a 3D grid
    """
    if not is_cube(n):
        raise ValueError('only works if n is a power of 3')
    
    L = .9 * boxsize
    # how many points can I put along once axis
    points_per_dim = int(np.round(np.cbrt(n)))
    dx = L / (points_per_dim - 1)
    
    # generate grid points here
    points = [np.array(ijk) * dx for ijk in itertools.product(range(points_per_dim), repeat=3)]
    return np.array(points) 

Here you can choose a pressure and density to simulate. You can try other values later if you wish. 

In [ ]:
pressure = 75  # Bar
density = 0.01
# standard name for files we use
fname = 'ar-sys'

The number of beads you are chosing will determine the final simulation box size together with the density

In [ ]:
n_beads = 125
box_volume = density_to_volume(density, n_beads)
box_length = np.cbrt(box_volume)
box = np.ones(3) * box_length

We start the simulations from a crystal. Therefore we need to place the beads on a crystal.

In [ ]:
u = make_universe(n_beads)
u.atoms.positions = grid_points(n_beads, box_length)
pdb = fname + '.pdb'
u.atoms.write(pdb)

Now we have everything to setup a system and create a CPLX using the `convert` tool from python.

In [ ]:
# Generate simply system description for convert
tops = {}
for i in range(n_beads):
    tops['AR-{}'.format(i)] = {'coordinate-file': pdb,
                               'domains': {
                                    'AR': {
                                        'selection': 'resnum {}'.format(i+1),
                                        'type': 'rigid'}}}
system = {'box': box.tolist(),
          'topology': tops}

# Generate and write CPLX
system = convert.convert(system, forcefield='argon.yaml')
with open(fname + '.cplx', 'w') as fh:
    yaml.dump(system, fh)

# Running an NPT Simulation

The last file needed for a simulation is the configuration where we select the NPT ensemble and set the volume fluctuation size. As the maximal delta for the volume moves  we use 10 % of the initial volume.

In [ ]:
config = {'structure': 'ar-sys.cplx',
          'montecarlo': {
              'algorithm': 'npt',
              'algorithm-params': {
                  'accept-func': 'metropolis',
                  'temperatur': '240',
                  'dV':  .1 * box_volume,
                  'pressure': pressure,
                  # turn on verbose logging of volume move acceptance rate
                  'verbose': False
              },
              'equilibration': 0,
              'seed': 4242,
              'short-range-cutoff': {
                  'enable': False
              }
          },
          'output': {
              'file': 'npt.xtc',
              'log': 'npt.log',
              'stat-file': 'npt.stat',
              'restart-freq': -1,
              # after how many sweeps is a structure written
              'freq': 2,
              # how many structures are written in total
              'nstructures': 100
          }
         }
with open('npt.conf', 'w') as fh:
    yaml.dump(config, fh)

Now we can run the simulation

In [ ]:
!complexes++ --config=npt.conf

# Analysis

## screen volume

Since we are particularly interested in the behaviour of the volume of our system, we want to extract the volume of the system over the simulation.
The following function uses MDAnalysis to do so.

In [ ]:
stat = pd.read_csv('npt.stat')

In [ ]:
fig, ax = plt.subplots()
ax.plot(stat.volume)
ax.set(xlabel ='frame', ylabel='volume [$\AA^3$]')
fig.tight_layout()

# Further tasks

1. calculate the resulting density of the simulation. The default number of structures is not enough for converged results!

2. How long to you need to run the simulation for converged results? Does this length depend on the chosen pressure?

3. How does your convergence behave when you use inconsistent starting densities for a given pressure. For example choose a high pressure and low density. Or a low pressure and high denisty. The equation of state diagram at the top should help you select pressure and density pairs.